## Imports

In [2]:
import folium, glob, geopy, tqdm
from geopy.geocoders import AlgoliaPlaces, Nominatim

import numpy as np
import pandas as pd

## Créer une carte des métiers

In [3]:
places = [place.split('/')[-1][:-4].replace('_', ' ') for place in sorted(glob.glob(
    '../../../Population/processing/1832_streets/*.png'))[:-3]]

nominatim = Nominatim(user_agent="projet_LTM")
algolia = AlgoliaPlaces(user_agent="projet_LTM")

places_lat, places_lon = [], []
for place in tqdm.tqdm(places):
    place_lat, place_lon = [], []
    for subplace in place.split('|'):
        for geocoder in [nominatim, algolia]:
            location = geocoder.geocode(f"{subplace}, Lausanne, Suisse")
            if not(location is None):
                place_lat.append(location.latitude)
                place_lon.append(location.longitude)
    places_lat.append(np.median(place_lat))
    places_lon.append(np.median(place_lon))

 53%|█████▎    | 20/38 [00:25<00:15,  1.15it/s]/Users/remipetitpierre/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/remipetitpierre/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 38/38 [00:40<00:00,  1.06s/it]


In [4]:
lausanne_map = folium.Map(location = [46.522, 6.635], zoom_start = 16, 
                          tiles='CartoDB positron', control_scale=True)

folium.raster_layers.ImageOverlay(
    image='https://gallica.bnf.fr/iiif/ark:/12148/btv1b530292533/f1/641,804,5564,6941/full/22/native.jpg',
    bounds=[[46.5014, 6.609], [46.538, 6.6563]],
    name='Plan Lausanne 1806',
    opacity=0.5,
    interactive=False,
    cross_origin=False,
    zindex=1,
).add_to(lausanne_map)

folium.LayerControl().add_to(lausanne_map)
lausanne_map

In [5]:
for place, lat, lon in zip(places, places_lat, places_lon):
    if not(np.isnan(lat)) and not(np.isnan(lon)):
        popup = folium.Popup(f'<center><b>{place}</b></center><br><img src="https://raw.githubusercontent.com/RPetitpierre/LTM_streets/master/{place.replace(" ", "_")}.png" height="20">', parse_html=False)
        marker = folium.Marker([lat, lon], popup = popup, marker_color='red').add_to(lausanne_map)
        
lausanne_map

In [6]:
lausanne_map.save("leaflet_wix.html")

## Créer un json à partir du fichier population 

In [31]:
population = pd.read_excel('../Population/recensements/1832_cleaned_data.xlsx', index_col=0)
population.head(3)

,NOMS DES RUES.,N° des MAISONS.,PROPRIÉTAIRES DES MAISONS.,NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME.,NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE.,Année de la naissance. [h],NOMS DE LEURS ÉPOUSES.,Année de la naissance. [f],Noms de leurs enfans actuellement dans la Commune.,Année de la naissance. [enf],...,Date de leur demeure dans la Commune.,VOCATION.,N° de leur Récépissé. [famille],"NOMS DES PENSIONNAIRES, COMMIS, APPRENTIS, OUVRIERS ET DOMESTIQUES. NOMS DE BAPTÊME.","NOMS DES PENSIONNAIRES, COMMIS, APPRENTIS, OUVRIERS ET DOMESTIQUES. NOMS DE FAMILLE.","D'où ils sont bourgeois, ou à ce défaut d'où ils sont originaires. [dom/pens]","Indiquer ici s'ils sont Pensionnaires, Commis, Apprentis, Ouvriers ou Domestiques.",N° de leur Récépissé. [dom/pens],OBSERVATIONS.,Page
0,[],['7'],['Gilliéron'],['Jean Louis'],"[['Deprez', 'Devez']]",['1768'],"[['Echs', 'Ochs']]",['1793'],"[['Laura', 'Lucia', 'Lucy']]",[],...,['1831'],"['journalier', 'journalier']",[],[],[],[],[],[],[],3
1,[],['7'],[],"[['Jean Andre', 'Jean Marc', 'Jean Marie', 'Je...",['Fontannaz'],['1778'],"['Sophprant', 'Loretti']",['1778'],"['Jeanne', 'Jeanne', 'Abram', 'Jeannette', ['D...","['1812', '1813', '1814', '1822']",...,['1812'],"['journalier', 'journalier']",['1525'],[],[],[],[],[],[],3
2,[],['7'],[],['Marie Louis'],"[['Guerry', 'Minery', 'Querry', 'Terry', 'Thie...",['1774'],['Chavan'],['1777'],"[['Louis', 'Louisa', 'Louise', 'Louison'], 'In...","['1816', '1813', '1826']",...,['né'],['vigneron'],[],[],[],[],[],[],[],3


In [29]:
population.to_json('population.json', orient='records')